In [93]:

addr = "http://127.0.0.1:5000"
payload = {
    "breadth": 1000,
    "scope": 100,
    "score_weight": 100,
    "llmModel": "gpt-4-1106-preview",
    "useQueryGeneration": False,
}
def set_payload(**to_set):
    for key, value in to_set.items():
        payload[key] = value

In [94]:
import requests

def run_query(
    query_input
) -> str:
    # Define the payload for the POST request
    set_payload(query=query_input)

    try:
        # Send the POST request
        response = requests.post(f"{addr}/query", json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        query_output = response.json().get("answer")
        return query_output
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [95]:
print(run_query(f"Return all circular saws"))

The circular saws mentioned are:

1. SwiftSaw Precision - A compact and lightweight circular saw designed for fine detailing and intricate work, appreciated by DIY hobbyists and artisans for its ease of maneuverability and exceptional accuracy.

2. CircuMax Pro-3000 - A high-performance circular saw designed for woodworking enthusiasts and professionals, known for its powerful motor and precision blade, offering reliability and versatility in cutting projects.

3. SolarBlade 4000 - A state-of-the-art solar-powered circular saw that combines efficiency with environmental sustainability, suitable for light to medium-duty tasks.

4. MegaCut TurboMax - A high-performance circular saw engineered for speed and power, featuring a turbocharged cutting mechanism and advanced cooling system for quick and efficient cutting tasks.

5. PrecisionBlade X4 - A high-precision circular saw designed for fine woodworking and crafting, featuring advanced laser-guided technology and customizable cutting pat

In [82]:
import json
names = ["drilling machines", "circle cutters", "circular saws"]
data = []
for name in names:
    for i in range(11):
        data.append({'type': name, 'query': run_query(f"Return all ${name} that consume at most ${i} volts and specify their consumption."), 'voltage': i})

In [83]:
with open('res.json', 'w') as file:
    json.dump(data, file, indent=4)  # indent=4 for pretty-printing

In [8]:
import json

import aas_core3.jsonization as aas_jsonization

with open('./../generation/example.json', 'r') as file:
    jsonable = json.load(file)

environment = aas_jsonization.environment_from_jsonable(
    jsonable
)

In [97]:
import aas_core3.types as aas_types

key_to_submodel = {}
name_to_aas={}
for submodel in environment.descend():
    if isinstance(submodel, aas_types.Submodel):
        key_to_submodel[submodel.id] = submodel
        # for submodel in aas.submodels:
        #     print(submodel.keys[0].value)
for aas in environment.descend():
    if isinstance(aas, aas_types.AssetAdministrationShell) and aas.asset_information.asset_kind != aas_types.AssetKind.INSTANCE:
        print(aas.id_short)
        res = {'name': aas.id_short}
        for submodel_key in aas.submodels:
            submodel = key_to_submodel[submodel_key.keys[0].value]
            if submodel.id_short == 'Energy Consumption':
                res['voltage'] = int(submodel.submodel_elements[0].value)
            elif submodel.id_short != 'Manufacturer':
                res['type'] = submodel.id_short
            submodel.id_short
        name_to_aas[aas.id_short]=res
        print(res)
        print()

QuantumPro X1
{'name': 'QuantumPro X1', 'type': 'Drilling', 'voltage': 3}

EcoDrive Elite
{'name': 'EcoDrive Elite', 'type': 'Drilling', 'voltage': 10}

TitanCraft Pro-1
{'name': 'TitanCraft Pro-1', 'type': 'Drilling', 'voltage': 6}

SwiftDrill Precision
{'name': 'SwiftDrill Precision', 'type': 'Drilling', 'voltage': 2}

SolarSprint 2000
{'name': 'SolarSprint 2000', 'type': 'Drilling', 'voltage': 7}

MegaForce Turbo
{'name': 'MegaForce Turbo', 'type': 'Drilling', 'voltage': 9}

GreenHarmony EcoPro
{'name': 'GreenHarmony EcoPro', 'type': 'Drilling', 'voltage': 9}

CirclerPro X2
{'name': 'CirclerPro X2', 'type': 'Circle Cutting', 'voltage': 2}

EcoRound Precision
{'name': 'EcoRound Precision', 'type': 'Circle Cutting', 'voltage': 2}

TitanCut CircleMaster
{'name': 'TitanCut CircleMaster', 'type': 'Circle Cutting', 'voltage': 8}

SwiftCircle Elite
{'name': 'SwiftCircle Elite', 'type': 'Circle Cutting', 'voltage': 10}

SolarSphere 3000
{'name': 'SolarSphere 3000', 'type': 'Circle Cutting',

In [96]:
import re
with open('./res.json', 'r') as file:
    query_answers = json.load(file)

to_actual={'drilling machines':"Drilling", 'circular saws': 'Sawing', 'circle cutters': 'Circle Cutting'}

# Find all voltage values
# voltage_values = re.findall(voltage_pattern, text)
def find_actual(typ, upper_bound_voltage):
    res = []
    for name, aas in name_to_aas.items():
        if (aas['type']==typ and aas['voltage']<=upper_bound_voltage):
            res.append(aas)
    return res

for query in query_answers:
    actual_type = to_actual[query['type']]
    name_pattern = r'\*\*(.*?)\*\*'

    # Regular expression to find the voltage value that comes after the name
    voltage_pattern = r'(\d+) volts'

    # Find all machine names
    machine_names = re.findall(name_pattern, query['query'])

    # Find all voltage values
    voltage_values = []
    if ('\n' in query['query']):
        # print('here')
        # print(query['query'].split('\n', 2))
        voltage_values = re.findall(voltage_pattern, query['query'].split('\n', 1)[1])

    # Combine names and their corresponding voltages
    machines_with_voltage = list(zip(machine_names, voltage_values))
    # print(actual_type)
    # print(machine_names)
    # print(voltage_values)
    print(query['query'])
    # if len(machines_with_voltage) > 0:
    #     print(type(machines_with_voltage[0][0]))
    additional = 0
    hit = 0
    wrong_voltage = 0
    actual = find_actual(actual_type, query['voltage'])
    for actual_aas in actual:
        for val in machines_with_voltage:
            machine, voltage = val
            if (actual_aas['name'] == machine):
                hit += 1
                if actual_aas['voltage'] != int(voltage):
                    wrong_voltage += 1
                machines_with_voltage.remove(val)
                break
    additional = len(machines_with_voltage)
    print('cur: ' + str(query['voltage']))
    print(f"{hit} of {len(actual)} {actual}")
    print(f"wrong voltage: {wrong_voltage}")
    print(f"additional: {additional}")
    # print(machines_with_voltage)
    print(machine_names)
    print(voltage_values)

    print()
    

Based on the provided information, there are no drilling machines listed that consume at most 0 volts. All the drilling machines mentioned have a specified energy consumption that requires a certain amount of voltage to operate, none of which is 0 volts.
cur: 0
0 of 0 []
wrong voltage: 0
additional: 0
[]
[]

Based on the provided information, there are no drilling machines listed that consume at most 1 volt. All the drilling machines mentioned have a higher voltage consumption requirement.
cur: 1
0 of 0 []
wrong voltage: 0
additional: 0
[]
[]

The drilling machines that consume at most 2 volts and their consumption are:

1. **SwiftDrill Precision** - It consumes 2 volts and 2 amperes.
cur: 2
1 of 1 [{'name': 'SwiftDrill Precision', 'type': 'Drilling', 'voltage': 2}]
wrong voltage: 0
additional: 0
['SwiftDrill Precision']
['2']

The drilling machines that consume at most 3 volts and their consumption details are:

1. **SwiftDrill Precision**
   - Required Voltage: 2 volts
   - Required 

In [107]:
import random

names = list(map(lambda aas: aas['name'], name_to_aas.values()))
# print(", ".join(random.sample(names, 5)))
data = []
for name in names:
    for iteration in range(10):
        for i in range(1, 11):
            machines = ", ".join(random.sample(names, i))
            data.append({'type': name, 'query': run_query(f"Get number of available {machines} machines. Specify the number for each machine separately."), 'voltage': i})

KeyboardInterrupt: 

In [ ]:
with open('test2.json', 'w') as file:
    json.dump(data, file, indent=4)  # indent=4 for pretty-printing